<a href="https://colab.research.google.com/github/sseydaltin/makine/blob/main/ridge_lasso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HITTERS Veri Seti

Hitter veri seti genellikle beyzbol oyuncularının performans verilerini içeren bir veri kümesidir. Bu veri seti, 1986 yılı itibariyle Amerikan beyzbol ligindeki oyunculara ait çeşitli özellikleri ve performanslarını içerir.

**Hikaye ve Amaç:**

Veri seti, oyuncuların maaş tahminlerini yapmayı amaçlayan bir çalışma için kullanılabilir. Amaç, oyuncunun performansını belirleyen faktörlerin (örneğin, yapılan home run'lar, vuruşlar, yürüyüşler vb.) maaşı üzerinde nasıl bir etkiye sahip olduğunu incelemektir. Bu, oyuncuların maaşlarının hangi faktörlere dayandığını anlamak ve buna bağlı olarak tahminler yapmak için kullanılabilir.

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np


Veri Yüklemesi

In [5]:
file_path = 'hitters_processed.pkl'
data = pd.read_pickle('/content/hitters_processed.pkl')

In [6]:
print(data.shape)
print(data.head())


(263, 39)
      AtBat      Hits     HmRun      Runs       RBI     Walks     Years  \
1 -0.602900 -0.595675 -0.529975 -1.207183 -0.522128 -0.096826  1.408945   
2  0.512542  0.492260  0.738195  0.442494  0.794576  1.614738 -0.905036   
3  0.628167  0.736490  0.968772  0.403216  1.026936 -0.189343  0.777859   
4 -0.562092 -0.462459 -0.184110 -0.618013 -0.367222 -0.513153 -1.115398   
5  1.294712  1.358167 -0.875840  0.756718 -0.018682 -0.281860  0.777859   

     CAtBat     CHits    CHmRun  ...  NEW_CHm_CAT  NEW_Diff_Atbat  \
1  0.373366  0.200587  0.109509  ...    -0.272565       -0.003834   
2 -0.460668 -0.418475  0.011934  ...     1.012006       -1.149114   
3  1.369180  1.412506  2.304938  ...     0.837941       -0.740787   
4 -1.021870 -1.001506 -0.817450  ...     0.431502        0.471475   
5  0.811634  0.688630 -0.703613  ...    -1.345732        1.085953   

   NEW_Diff_Hits  NEW_Diff_HmRun  NEW_Diff_Runs  NEW_Diff_RBI  NEW_Diff_Walks  \
1       0.092772       -0.280053      -0.47

**Hedef ve Özelliklerin Ayrılması**

In [7]:
X = data.drop(columns=['Salary'])  # Bağımsız değişkenler
y = data['Salary']                 # Bağımlı değişken (hedef)

**Eğitim ve Test Verisi Ayrımı**

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

**Lineer Regresyon Modeli Kurma ve Eğitme**

In [9]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Modeli oluştur
lr_model = LinearRegression()

# Modeli eğit
lr_model.fit(X_train, y_train)

# Test verisi üzerinde tahmin yap
y_pred = lr_model.predict(X_test)

# MSE ve R2 skorlarını hesapla
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Lineer Regresyon - MSE: {mse:.2f}")
print(f"Lineer Regresyon - R2 Skoru: {r2:.2f}")


Lineer Regresyon - MSE: 42335.52
Lineer Regresyon - R2 Skoru: 0.70


**Sonuçların Yorumu**

**📉 Mean Squared Error (MSE):** 42335.52

Bu, tahminlerin ne kadar sapma gösterdiğini gösterir.

Ortalama olarak modelin yaptığı tahminlerin kare farkı 42.335 civarında. MSE değeri bağlamdan bağımsız tek başına çok anlamlı olmasa da, diğer modellerle karşılaştırma için idealdir.


Özellikle maaş gibi parasal bir değeri tahmin ederken 42 bin dolarlık bir hata görece büyük olabilir — ama verinin ölçeğine bağlı.


**📈 R² Skoru:** 0.70

Bu, modelin bağımlı değişkendeki (Salary) değişkenliğin %70’ini açıkladığını gösterir.


%70'lik açıklama oranı istatistiksel olarak gayet kabul edilebilir. Ancak %30'luk kısmın açıklanmadığını da unutmamalıyız.


Daha yüksek doğruluk isteniyorsa Ridge veya Lasso gibi düzenlileştirme teknikleriyle model iyileştirilebilir.

**Ridge ve Lasso Hatırlatma**

idge ve Lasso regresyon, Lineer Regresyon modeline benzer ancak düzenlileştirme (regularization) tekniği kullanarak aşırı öğrenmeyi (overfitting) önlemeyi amaçlar. Özellikle çok sayıda öznitelik (feature) olduğunda veya bunlar arasında çoklu doğrusal bağlantı (multicollinearity) varsa oldukça işe yararlar.

**Ridge Regresyon (L2 Regularization)**

Temel fikir: Katsayıların (coef) karelerinin toplamına bir ceza ekler.

Bu ceza, çok büyük katsayıların değerlerini küçülterek modelin karmaşıklığını azaltır.

Hiçbir katsayıyı sıfırlamaz, sadece küçültür.

Multicollinearity varsa Ridge, katsayıları dengeleyerek daha istikrarlı tahminler üretir.

 **Lasso Regresyon (L1 Regularization)**

Temel fikir: Katsayıların mutlak değerlerinin toplamına ceza uygular.

Bu sayede bazı katsayıları tamamen sıfıra indirir → özellik seçimi (feature selection) gibi davranır.

Gereksiz değişkenleri modelden çıkarabilir.

Özellikle yüksek boyutlu veri setlerinde kullanışlıdır.

**Ridge Regresyon**

In [10]:
from sklearn.linear_model import Ridge

ridge_model = Ridge(alpha=1.0)  # alpha = düzenlileştirme parametresi
ridge_model.fit(X_train, y_train)

y_pred_ridge = ridge_model.predict(X_test)

mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)

print(f"Ridge Regresyon - MSE: {mse_ridge:.2f}")
print(f"Ridge Regresyon - R2 Skoru: {r2_ridge:.2f}")


Ridge Regresyon - MSE: 48202.13
Ridge Regresyon - R2 Skoru: 0.66


**Yorum:**

MSE'nin Lineer Regresyona göre arttığını görüyoruz
(42,335 → 48,202). Bu, modelin tahmin hatalarının biraz daha büyüdüğünü gösteriyor.

R² skoru da düştü (0.70 → 0.66), yani model açıklayıcılığını biraz kaybetmiş.

**Lasso Regresyon**

In [11]:
from sklearn.linear_model import Lasso

lasso_model = Lasso(alpha=1.0)  # alpha = düzenlileştirme katsayısı
lasso_model.fit(X_train, y_train)

y_pred_lasso = lasso_model.predict(X_test)

mse_lasso = mean_squared_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)

print(f"Lasso Regresyon - MSE: {mse_lasso:.2f}")
print(f"Lasso Regresyon - R2 Skoru: {r2_lasso:.2f}")


Lasso Regresyon - MSE: 51137.66
Lasso Regresyon - R2 Skoru: 0.64


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.562e+03, tolerance: 3.462e+03
  model = cd_fast.enet_coordinate_descent(


**Yorum:**

Lasso'nun MSE değeri Ridge ve Lineer regresyona göre daha yüksek. Bu, modelin tahmin hatalarının biraz daha fazla olduğunu gösteriyor.

R² skoru da en düşük seviyede (0.64), yani maaş değişkenindeki varyansın sadece %64’ünü açıklayabiliyor.

**ALPHA DEĞERİ BULMA**

Nasıl Alpha Değeri Bulunur?

✅ Cross-validation (Çapraz Doğrulama) yöntemi kullanılır.

Amaç: alpha değerini sistematik şekilde test etmek.

Kullanılan araçlar: RidgeCV ve LassoCV (Scikit-learn’de hazır olarak geliyor).

Bu sınıflar, farklı alpha değerlerini dener ve en düşük hata veren değeri seçer.

In [12]:
from sklearn.linear_model import RidgeCV, LassoCV

# Denenecek alpha değerleri
alphas = np.logspace(-3, 3, 100)

# Ridge için en iyi alpha
ridge_cv = RidgeCV(alphas=alphas, store_cv_values=True)
ridge_cv.fit(X_train, y_train)
best_alpha_ridge = ridge_cv.alpha_

# Lasso için en iyi alpha (5 katlı CV)
lasso_cv = LassoCV(alphas=alphas, cv=5, max_iter=10000)
lasso_cv.fit(X_train, y_train)
best_alpha_lasso = lasso_cv.alpha_

print(f"En iyi Ridge alpha: {best_alpha_ridge:.4f}")
print(f"En iyi Lasso alpha: {best_alpha_lasso:.4f}")


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_ridge.py:2385: FutureWarning: 'store_cv_values' is deprecated in version 1.5 and will be removed in 1.7. Use 'store_cv_results' instead.
  warnings.warn(


En iyi Ridge alpha: 4.3288
En iyi Lasso alpha: 2.1544


**Modelleri En İyi Alpha ile Tekrar Eğitme**

In [13]:
# Ridge yeniden eğitme
ridge_best = Ridge(alpha=best_alpha_ridge)
ridge_best.fit(X_train, y_train)
y_pred_ridge_best = ridge_best.predict(X_test)
mse_ridge_best = mean_squared_error(y_test, y_pred_ridge_best)
r2_ridge_best = r2_score(y_test, y_pred_ridge_best)

# Lasso yeniden eğitme
lasso_best = Lasso(alpha=best_alpha_lasso, max_iter=10000)
lasso_best.fit(X_train, y_train)
y_pred_lasso_best = lasso_best.predict(X_test)
mse_lasso_best = mean_squared_error(y_test, y_pred_lasso_best)
r2_lasso_best = r2_score(y_test, y_pred_lasso_best)

print(f"En iyi Ridge - MSE: {mse_ridge_best:.2f}, R2: {r2_ridge_best:.2f}")
print(f"En iyi Lasso - MSE: {mse_lasso_best:.2f}, R2: {r2_lasso_best:.2f}")


En iyi Ridge - MSE: 56973.88, R2: 0.59
En iyi Lasso - MSE: 62017.82, R2: 0.56


** Yorum:**

Ridge Modeli:

MSE: Önceki modelden (48,202) daha yüksek.

R² Skoru: Daha düşük (0.66 → 0.59).

Sonuç: Ridge'in en iyi alpha değeri ile model performansında düşüş gözlemleniyor. Bu, düzenlileştirmenin (regularization) modelin doğruluğunu pekiştirmek yerine fazla sadeleştirdiğini gösteriyor.

Lasso Modeli:

MSE: Önceki modelden (51,137) daha yüksek.

R² Skoru: Yine düşük (0.64 → 0.56).

Sonuç: Lasso'nun performansındaki düşüş, büyük ihtimalle bazı önemli değişkenlerin sıfırlanmasından kaynaklanıyor. Lasso, özellik seçiminde daha agresif olduğu için bazen önemli değişkenleri çıkarabiliyor.

**Polinom Özellikler Ekleyerek Modeli Yeniden Değerlendirme**

1- PolynomialFeatures kullanarak bağımsız değişkenleri dönüştüreceğiz.

2- Bu dönüştürülmüş veriyi kullanarak modelleri eğiteceğiz.

**Polinom Özellikler Ekleyip Modelleri Eğitme**

In [14]:
from sklearn.preprocessing import PolynomialFeatures

# Polinom özellik ekleme (derece 2)
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train)
X_test_poly = poly.transform(X_test)

# Ridge Modeli (Polinom özelliklerle)
ridge_poly = Ridge(alpha=best_alpha_ridge)
ridge_poly.fit(X_train_poly, y_train)
y_pred_ridge_poly = ridge_poly.predict(X_test_poly)
mse_ridge_poly = mean_squared_error(y_test, y_pred_ridge_poly)
r2_ridge_poly = r2_score(y_test, y_pred_ridge_poly)

# Lasso Modeli (Polinom özelliklerle)
lasso_poly = Lasso(alpha=best_alpha_lasso, max_iter=10000)
lasso_poly.fit(X_train_poly, y_train)
y_pred_lasso_poly = lasso_poly.predict(X_test_poly)
mse_lasso_poly = mean_squared_error(y_test, y_pred_lasso_poly)
r2_lasso_poly = r2_score(y_test, y_pred_lasso_poly)

# Sonuçları yazdır
print(f"Polinom Özelliklerle Ridge - MSE: {mse_ridge_poly:.2f}, R2: {r2_ridge_poly:.2f}")
print(f"Polinom Özelliklerle Lasso - MSE: {mse_lasso_poly:.2f}, R2: {r2_lasso_poly:.2f}")


Polinom Özelliklerle Ridge - MSE: 77190.35, R2: 0.45
Polinom Özelliklerle Lasso - MSE: 69779.92, R2: 0.50


**Yorum:**

MSE'nin arttığını ve R² skorlarının düştüğünü görüyoruz.

Ridge ve Lasso modellerinin MSE'leri, polinom özellikler eklenmeden önceki haline göre çok daha yüksek oldu.

R² skorları da azaldı (0.59 → 0.45 ve 0.56 → 0.50).

Polinom özellikler eklemek, modelin doğrusal olmayan ilişkileri öğrenmesine yardımcı olabilir, ancak burada modelin karmaşıklığını artırdı ve aşırı öğrenmeye (overfitting) yol açtı.

# NELER YAPTIK?



1. **Veri Yükleme**: Modellemeye hazır veriyi yükledik ve hedef değişkeni `Salary` olarak belirledik.
2. **Lineer Regresyon**: İlk olarak lineer regresyon modeli eğittik:
   - **MSE**: `42,335.52`, **R²**: `0.70`.
3. **Ridge ve Lasso Modelleri**: Ridge ve Lasso regresyonları ile düzenlileştirme yaptık:
   - **Ridge**: MSE: `48,202.13`, R²: `0.66`
   - **Lasso**: MSE: `51,137.66`, R²: `0.64`
4. **En İyi Alpha Değerleri**: Ridge ve Lasso için en uygun `alpha` değerlerini bulduk:
   - **Ridge**: MSE: `56,973.88`, R²: `0.59`
   - **Lasso**: MSE: `62,017.82`, R²: `0.56`
5. **Polinom Özellikler**: Polinom özellikler ekleyerek modelleri yeniden eğittik:
   - **Ridge**: MSE: `77,190.35`, R²: `0.45`
   - **Lasso**: MSE: `69,779.92`, R²: `0.50`

### Sonuç:
- **Lineer Regresyon** en iyi performansı verdi.
- **Ridge ve Lasso** ile düzenlileştirme, doğruluğu biraz düşürse de aşırı öğrenmeyi engellemeye yardımcı olabilir.
- **Polinom özellikler** eklemek modelin karmaşıklığını artırdı, ancak performans düşüşüne neden oldu.
